# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [12]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import os
import joblib

ImportError: cannot import name 'clean_data' from 'train' (C:\Users\Sara\Documents\GitHub\nd00333-capstone\starter_file\train.py)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'labor-prediction-experiment'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [24]:
from train import clean_data
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.8)

train = pd.concat([x_train, y_train], axis=1)

True


ImportError: cannot import name 'clean_data' from 'train' (C:\Users\Sara\Documents\GitHub\nd00333-capstone\starter_file\train.py)

In [ ]:
# TODO: Put your automl settings here
#automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
experiment_timeout_minutes=30,
    task='regression',
    primary_metric='RMSE',
    training_data=train,
    label_column_name='y',
    n_cross_validations=10)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_mlrun, fitted_mlmodel = automl_run.get_output()
print(best_mlrun)
print(fitted_mlmodel)

In [ ]:
joblib.dump(fitted_mlmodel, "automlmodel.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice
from azureml.core.model import Model

description = 'Best AutoML Model'
model = best_mlrun.register_model(description = description,
                               tags=None})

print(best_mlrun.model_id)


#from azureml.core.model import Model

#model = Model.register(workspace = ws,
                        model_path ="labor-prediction.pkl",
                        model_name = "automl_pred")


env = Environment.get(workspace, "AzureML").clone(env_name)

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

deployment_config = LocalWebservice.deploy_configuration()

model = Model(ws, name='automl_pred')
service = Model.deploy(ws, 'predictionservice', [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

scoring_uri = service.scoring_uri
headers = {'Content-Type':'application/json'}

data = {"data":  [ [67,'ALUMINUM','DOOR',24,0,'SNAP BEADS',9,118] } 

response = requests.post(scoring_uri, data=test_data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)
    
service.delete()